<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/PyCaret_%EC%B5%9C%EB%B9%88%EA%B0%92%EB%8C%80%EC%B2%B4_GWAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ 라이브러리 설치
!pip install pycaret lightgbm xgboost catboost openpyxl --quiet
!pip install numpy==1.24.4 --force-reinstall --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.

In [ ]:
# ✅ Colab에 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving 2016년 표현형 데이터.xlsx to 2016년 표현형 데이터.xlsx


In [ ]:
from pycaret.regression import setup, compare_models, predict_model, pull
from scipy.stats import pearsonr
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 1. 데이터 로드
geno_path = "유전형인코딩_최빈값.csv"
pheno_path = "2016년 표현형 데이터.xlsx"

geno_df = pd.read_csv(geno_path, index_col=0)
pheno_df = pd.read_excel(pheno_path, index_col=0)

trait_map = {
    "과중": "과중 (g)",
    "과장": "과장 (mm)",
    "과폭": "과폭 (mm)",
    "과피두께": "과피두께 (mm)",
    "당도": "당도 (%)",
    "과실경도": "과실경도 (kg)"
}

# 2. SNP 조건
snp_files = {
    "Top 100": "GWAS_SNP(100).csv",
    "Top 500": "GWAS_SNP(500).csv",
    "Top 1000": "GWAS_SNP(1000).csv",
    "Top 3000": "GWAS_SNP(3000).csv",
    "Top 5000": "GWAS_SNP(5000).csv",
    "Top 10000": "GWAS_SNP(10000).csv",
}

results = []



for label, file_path in snp_files.items():
    snp_df = pd.read_csv(file_path)
    traits = snp_df["Trait"].unique()

    for trait in traits:
        try:
            # ✅ 이름 매핑
            true_trait = trait_map.get(trait)
            if true_trait not in pheno_df.columns:
                raise ValueError(f"{true_trait} is not in phenotype DataFrame")

            top_snps = snp_df[snp_df["Trait"] == trait]["SNP"].tolist()
            geno_subset = geno_df.loc[top_snps].T
            geno_subset["target"] = pheno_df[true_trait]

            # PyCaret 설정
            setup(data=geno_subset, target="target", session_id=42, verbose=False)
            best_models = compare_models(include=["rf", "xgboost", "lightgbm", "catboost"], n_select=4)
            result_df = pull()

            # 모델별 PCC 수동 계산
            pcc_list = []
            for model in best_models:
                pred_df = predict_model(model)
                pcc, _ = pearsonr(pred_df["target"], pred_df["prediction_label"])
                pcc_list.append(pcc)

            # 결과 저장
            result_df["PCC"] = pcc_list
            result_df["SNP 개수"] = label
            result_df["표현형"] = true_trait
            results.append(result_df)

        except Exception as e:
            print(f"⚠️ 오류 발생 - 표현형: {trait}, SNP: {label}, 원인: {e}")
            continue


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,28.0573,1700.0482,40.6534,0.3745,0.5557,0.5627,0.4110
rf,Random Forest Regressor,28.1580,1710.0967,40.7906,0.3256,0.5265,0.5513,1.2270
catboost,CatBoost Regressor,29.3512,1777.9285,41.7154,0.2751,0.5303,0.5354,2.1410
xgboost,Extreme Gradient Boosting,29.9666,1949.4104,43.1768,0.2484,0.5400,0.5142,0.1600


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,27.3343,1529.0541,39.1031,0.4185,0.5895,0.6765


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,27.8061,1678.6073,40.9708,0.3616,0.6262,0.7609


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,29.5506,1866.6902,43.2052,0.2901,0.6292,0.7572


...

Processing:   0%|          | 0/24 [00:00<?, ?it/s]

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.5340,0.4664,0.6830,0.7319,0.1309,0.1321


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.5813,0.5210,0.7218,0.7005,0.1387,0.1447


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,0.6354,0.6692,0.8180,0.6153,0.1534,0.1542


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.5405,0.5027,0.7090,0.7110,0.1338,0.1317


In [ ]:
# 3. 결과 저장
if results:
    final_df = pd.concat(results, ignore_index=True)
    final_df.to_csv("모델성능비교_PCC_테이블.csv", index=False)
    print("✅ 결과 저장 완료 → 모델성능비교_PCC_테이블.csv")
    display(final_df.head(10))
else:
    print("❌ 저장할 결과가 없습니다.")


✅ 결과 저장 완료 → 모델성능비교_PCC_테이블.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
0,Light Gradient Boosting Machine,28.0573,1700.0482,40.6534,0.3745,0.5557,0.5627,0.411,0.667617,Top 100,과중 (g)
1,Random Forest Regressor,28.1580,1710.0967,40.7906,0.3256,0.5265,0.5513,1.227,0.666252,Top 100,과중 (g)
2,CatBoost Regressor,29.3512,1777.9285,41.7154,0.2751,0.5303,0.5354,2.141,0.632103,Top 100,과중 (g)
3,Extreme Gradient Boosting,29.9666,1949.4104,43.1768,0.2484,0.5400,0.5142,0.160,0.654376,Top 100,과중 (g)
4,Light Gradient Boosting Machine,6.1531,72.2097,8.1278,0.3678,0.1694,0.1382,0.424,0.617707,Top 100,과장 (mm)
5,Random Forest Regressor,6.3669,81.6821,8.6776,0.2531,0.1834,0.1459,0.572,0.718337,Top 100,과장 (mm)
6,CatBoost Regressor,6.5826,84.4209,8.8956,0.2192,0.1900,0.1522,2.042,0.678978,Top 100,과장 (mm)
7,Extreme Gradient Boosting,7.5126,106.3532,9.9441,-0.0258,0.2113,0.1730,0.181,0.607132,Top 100,과장 (mm)
8,Light Gradient Boosting Machine,8.0700,115.5728,10.5643,0.4490,0.2043,0.1707,0.399,0.747042,Top 100,과폭 (mm)
9,Random Forest Regressor,7.9064,119.4232,10.6298,0.4317,0.1998,0.1637,0.270,0.731374,Top 100,과폭 (mm)


In [ ]:
import pandas as pd

# 1. 기존 결과 불러오기
df = pd.read_csv("모델성능비교_PCC_테이블.csv")

# 2. 표현형 + SNP 수별 R² 최고 모델만 추출
ranking_df = df.loc[df.groupby(["표현형", "SNP 개수"])["R2"].idxmax()]

# 3. 결과 저장
ranking_df.to_csv("모델성능비교_PCC__랭킹요약.csv", index=False)

# 4. 미리보기
ranking_df.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
16,Light Gradient Boosting Machine,0.0845,0.0111,0.1045,0.2082,0.0650,0.1454,0.521,0.577366,Top 100,과실경도 (kg)
64,Random Forest Regressor,0.0792,0.0111,0.1018,0.2516,0.0635,0.1361,1.415,0.668258,Top 1000,과실경도 (kg)
136,CatBoost Regressor,0.0770,0.0098,0.0967,0.3370,0.0603,0.1348,185.661,0.640816,Top 10000,과실경도 (kg)
88,Random Forest Regressor,0.0783,0.0104,0.0995,0.2879,0.0622,0.1370,4.483,0.657378,Top 3000,과실경도 (kg)
40,CatBoost Regressor,0.0797,0.0106,0.1015,0.2492,0.0632,0.1362,9.912,0.519102,Top 500,과실경도 (kg)


In [ ]:
# 📌 Colab 전용 성능 요약 코드 (R² 기준 요약 3종)

import pandas as pd

# # 👉 파일 업로드
# from google.colab import files
# uploaded = files.upload()  # 모델성능비교_GWAS_전체정리.csv 업로드

# 👉 파일 로드
df = pd.read_csv("모델성능비교_PCC_테이블.csv")

# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["R2"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 표현형별 최고 R² 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 R²")
display(mean_r2_by_snp)

print("📌 모델별 평균 R²")
display(mean_r2_by_model)

📌 표현형별 최고 R² 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
0,CatBoost Regressor,0.0770,0.0098,0.0967,0.3370,0.0603,0.1348,185.661,0.640816,Top 10000,과실경도 (kg)
1,Light Gradient Boosting Machine,6.1615,71.0723,8.0484,0.3855,0.1711,0.1417,0.895,0.673737,Top 500,과장 (mm)
2,Random Forest Regressor,23.8346,1161.4408,33.7983,0.5388,0.4879,0.5393,4.822,0.647283,Top 3000,과중 (g)
3,CatBoost Regressor,7.0471,94.5386,9.6374,0.5143,0.1950,0.1555,192.499,0.684356,Top 10000,과폭 (mm)
4,Random Forest Regressor,0.5545,0.5373,0.7199,0.2003,0.1272,0.1238,3.266,0.715682,Top 3000,과피두께 (mm)
5,CatBoost Regressor,0.6248,0.6597,0.8053,0.3731,0.1557,0.1551,99.760,0.850080,Top 5000,당도 (%)


📌 SNP 개수별 평균 R²


,SNP 개수,평균 R²
0,Top 100,0.157721
1,Top 1000,0.253517
2,Top 10000,0.292417
3,Top 3000,0.286342
4,Top 500,0.268142
5,Top 5000,0.297029


📌 모델별 평균 R²


,Model,평균 R²
0,CatBoost Regressor,0.301478
1,Extreme Gradient Boosting,0.155872
2,Light Gradient Boosting Machine,0.266747
3,Random Forest Regressor,0.312681


In [ ]:
import pandas as pd

# 👉 파일 로드
df = pd.read_csv("모델성능비교_PCC_테이블.csv")


In [ ]:
# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["PCC"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["PCC"].mean().reset_index().rename(columns={"PCC": "평균 PCC"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["PCC"].mean().reset_index().rename(columns={"PCC": "평균 PCC"})

# 👉 결과 출력
print("📌 표현형별 최고 PCC 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 PCC")
display(mean_r2_by_snp)

print("📌 모델별 평균 PCC")
display(mean_r2_by_model)


📌 표현형별 최고 PCC 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),PCC,SNP 개수,표현형
0,Random Forest Regressor,0.0792,0.0111,0.1018,0.2516,0.0635,0.1361,1.415,0.668258,Top 1000,과실경도 (kg)
1,CatBoost Regressor,6.2900,77.7481,8.5277,0.2837,0.1802,0.1454,18.725,0.748046,Top 1000,과장 (mm)
2,Extreme Gradient Boosting,26.9979,1441.1549,37.4811,0.4111,0.5042,0.4978,0.581,0.727994,Top 500,과중 (g)
3,Light Gradient Boosting Machine,7.5582,107.6044,10.2468,0.4650,0.2018,0.1611,1.347,0.792076,Top 1000,과폭 (mm)
4,Random Forest Regressor,0.5527,0.5715,0.7427,0.1412,0.1323,0.1251,12.766,0.757993,Top 10000,과피두께 (mm)
5,Random Forest Regressor,0.6574,0.7474,0.8475,0.2718,0.1650,0.1660,0.746,0.879026,Top 500,당도 (%)


📌 SNP 개수별 평균 PCC


,SNP 개수,평균 PCC
0,Top 100,0.658276
1,Top 1000,0.722605
2,Top 10000,0.691988
3,Top 3000,0.697787
4,Top 500,0.700554
5,Top 5000,0.690268


📌 모델별 평균 PCC


,Model,평균 PCC
0,CatBoost Regressor,0.704761
1,Extreme Gradient Boosting,0.653876
2,Light Gradient Boosting Machine,0.699131
3,Random Forest Regressor,0.716550


In [ ]:
# ✅ 4. SNP 개수별 × 모델별 평균 R² 계산
mean_r2_by_snp_and_model = df.groupby(["SNP 개수", "Model"])["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 SNP 개수별 × 모델별 평균 R²")
display(mean_r2_by_snp_and_model)


📌 SNP 개수별 × 모델별 평균 R²


,SNP 개수,Model,평균 R²
0,Top 100,CatBoost Regressor,0.160417
1,Top 100,Extreme Gradient Boosting,0.018300
2,Top 100,Light Gradient Boosting Machine,0.258083
3,Top 100,Random Forest Regressor,0.194083
4,Top 1000,CatBoost Regressor,0.297550
5,Top 1000,Extreme Gradient Boosting,0.123150
6,Top 1000,Light Gradient Boosting Machine,0.284067
7,Top 1000,Random Forest Regressor,0.309300
8,Top 10000,CatBoost Regressor,0.371133
9,Top 10000,Extreme Gradient Boosting,0.211017


In [ ]:
# ✅ 피벗 테이블 형태로 보기 좋게 변환
pivot_r2_by_snp_and_model = mean_r2_by_snp_and_model.pivot(index="SNP 개수", columns="Model", values="평균 R²")

# 👉 결과 출력
print("📌 SNP 개수별 × 모델별 평균 R² (피벗 테이블)")
display(pivot_r2_by_snp_and_model)

📌 SNP 개수별 × 모델별 평균 R² (피벗 테이블)


Model,CatBoost Regressor,Extreme Gradient Boosting,Light Gradient Boosting Machine,Random Forest Regressor
SNP 개수,,,,
Top 100,0.160417,0.018300,0.258083,0.194083
Top 1000,0.297550,0.123150,0.284067,0.309300
Top 10000,0.371133,0.211017,0.249450,0.338067
Top 3000,0.329967,0.202117,0.251033,0.362250
Top 500,0.297300,0.171483,0.285000,0.318783
Top 5000,0.352500,0.209167,0.272850,0.353600
